In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import math
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import shuffle
from keras import callbacks
import pickle


Using TensorFlow backend.


In [3]:
PATH = r"../../Data/completeLoad.csv"
df = pd.read_csv(PATH)
temp = df.set_index(['zone_id', 'year', 'month', 'day'])
colname_to_int = {}
for i in list(temp.columns):
    colname_to_int[i] = int(i[1:])

temp = temp.rename(columns=colname_to_int)
temp = temp.stack()
temp = temp.unstack(level=0)
temp.columns.names = [None]
temp.index.names = [None, None, None, None]
df_zone = temp
temp.head(2)

1         2         3      4       5         6         7   \
2004 1 1 1  16853.0  126259.0  136233.0  484.0  6829.0  133088.0  136233.0   
         2  16450.0  123313.0  133055.0  457.0  6596.0  129909.0  133055.0   

                8        9        10       11        12       13       14  \
2004 1 1 1  3124.0  75243.0  23339.0  90700.0  118378.0  20673.0  21791.0   
         2  2956.0  67368.0  22100.0  86699.0  112480.0  19666.0  21400.0   

                 15       16       17        18       19       20  
2004 1 1 1  65970.0  28752.0  30645.0  200946.0  82298.0  79830.0  
         2  64600.0  27851.0  30461.0  195835.0  79827.0  77429.0